In [1]:
#interface graphique 
from tkinter import Tk, Canvas, Event, Button
import pprint
import random
import time
#import de notebook
import nbimporter
from ia import Ia_dumb
Ia_dumb()

<h2><b>GAME</b></h2>

In [2]:
#function pour dessiner 
def pack_util(p) : 
    return p.pack()

In [3]:
#classe principale 
class Snake_game :
    def __init__(self, tinker, size = 500, play_btn = True):
        #initialisation des variables
        self.game_size = size
        self.pixel_size = 20
        self.tinker = tinker
        self.commands = {
            "z" : "top",
            "q" : "left",
            "d" : "right",
            "s" : "bottom"
        }
        self.direction = ""
        self.commands_history = [self.direction]
        
        #initialisation fenêtre tkinter
        self.tinker.title("Snake")
        self.tinker.resizable(False, False)
        self.tinker.tk.call("tk", "scaling", 4.0)
        
        #initialisation de la matrice / creation d'une ligne vide
        self.grid_pos = [[] for i in range(0, int(self.game_size / self.pixel_size))]
        self.create_grid()
        
        #booleen pour afficher le bouton play 
        if play_btn : 
            #bouton jouer
            self.play_button = Button(self.tinker, text = "PLAY", command = self.play, width = 8, height = 2)
            pack_util(self.play_button)
            
    def create_canvas(self) :
        #canvas
        canvas = Canvas(self.tinker, width = self.game_size, height = self.game_size, background = "black", highlightthickness = 0)
        return canvas
          
    def create_grid(self) :
        
        #initialisation d'un dictionnaire
        pos_dict = {}             
        #remplir chaque ligne avec une liste de tuple de coordonnées possibles  
        for x in range(0, self.game_size + 1, self.pixel_size) :
            pos_dict[x] = []
            for y in range(0, self.game_size + 1, self.pixel_size) :
                pos_dict[x].append((x, y))

        #création de matrice      
        for x in list(pos_dict.keys())[:-1]:
            for i in range(len(pos_dict[x])-1):
                self.grid_pos[i].append([pos_dict[x][i],pos_dict[x+self.pixel_size][i+1]])
        
    #fonction d'écoute de touche    
    def set_key_event(self, event: Event, canvas) :
        #vérifier la touche 
        if event.char in self.commands :
            #définir la direction
            self.direction = self.commands[event.char]
            
            #on vérifie si l'utilisateur n'appuie pas deux fois sur la même touche
            if self.commands_history[len(self.commands_history) - 2] != self.direction :
                
                self.commands_history.append(self.direction)
                self.snake.move(self.direction, canvas, self.commands_history)
        
    def play(self) :
        print("Playing")
        #retirer le bouton play 
        self.play_button.destroy()
        
        #initialisation canvas
        self.canvas = self.create_canvas()
        
        #on vérifie si il y a plus de 1 élément fils sur la fenêtre
        if len(self.tinker.winfo_children()) == 1 :
            #initialisation de la pomme
            self.apple = Apple(self.tinker, self.game_size, False)
            self.apple.create_apple(self.canvas)
            
            #initialisation du snake
            self.snake = Snake(self.tinker, self.game_size, False)
            self.snake.create_snake(self.canvas)
            
            #écouter touche du clavier
            self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
            
            pack_util(self.canvas)

In [4]:
#TODO set Apple 
class Apple (Snake_game):
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.init_pos = int((self.game_size/self.pixel_size)/2)

    def create_apple(self, canvas):
        print("apple", canvas)
        #création de la pomme
        self.apple = canvas.create_rectangle(
                self.grid_pos[self.init_pos][self.init_pos][0][1],
                self.grid_pos[self.init_pos][self.init_pos][0][0],
                self.grid_pos[self.init_pos][self.init_pos][1][1],
                self.grid_pos[self.init_pos][self.init_pos][1][0],
                outline="#000000",
                fill="#ff8f26"
            )

In [5]:
class Snake (Snake_game) :
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        #initialisation position snake
        self.snake_positions = [(5,5), (6,5), (7, 5)]
        self.direction = "none"
        
    def create_snake(self, canvas):
        print("snake", canvas)
        for x_position, y_position in self.snake_positions :
            self.snake_part = canvas.create_rectangle(
                self.grid_pos[x_position][y_position][0][1],
                self.grid_pos[x_position][y_position][0][0],
                self.grid_pos[x_position][y_position][1][1],
                self.grid_pos[x_position][y_position][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake')
            )
            
    def move(self, direction, canvas, commands_history) :
        movement = True
        #changement de direction
        direction_is_diff = False
        #boucle de mouvement
        while movement : 
            for i, snake_id in enumerate(canvas.find_withtag("snake")):
                #coordonné d'une partie du snake par id
                coords = canvas.coords(snake_id)
                
                draw = False

                x = self.snake_positions[i][0]
                y = self.snake_positions[i][1]
                
                all_x = [j[0] for j in self.snake_positions]
                all_y = [k[1] for k in self.snake_positions]
                
                max_x = max(all_x)
                max_y = max(all_y)
                
                #mouvement à droite
                if direction is "right" :
                    y = max_y
                    #on vérifie que ça ne touche pas les bords 
                    if x < len(self.grid_pos[x]) - 1:
                        if direction_is_diff:
                            x += i+1
                            if i is len(canvas.find_withtag("snake")) - 1 :
                                direction_is_diff = False
                                print("changed is diff", direction_is_diff)
                        else :
                            x += 1
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                
                if direction is "left" :
                    y = max_y
                    if x < len(self.grid_pos[x]) - 1:
                        if x > 0 :
                            if direction_is_diff :
                                x -= i+1
                                if i is len(canvas.find_withtag("snake")) - 1 :
                                    direction_is_diff = False
                                    print("changed is diff", direction_is_diff)
                            else :
                                x -= 1

                            self.snake_positions[i] = (x,y)
                            draw = True
                        else :
                            movement = False
                            break
                    else :
                        movement = False
                        break
                        
                if direction is "bottom" :
                    if y < len(self.grid_pos) - 1:
                        x = self.snake_positions[0][0]
                        if direction_is_diff is False:
                            y += i+1
                            if i is len(canvas.find_withtag("snake")) - 1 :
                                direction_is_diff = True
                                print("changed is diff", direction_is_diff)
                        else :
                            y += 1
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                print(f"Snake_part-{i} : {x, y}, direction : {direction}, direction_is_diff : {direction_is_diff}")
                #dessin du snake
                if draw : 
                    canvas.coords(
                        snake_id,
                        self.grid_pos[x][y][0][1],
                        self.grid_pos[x][y][0][0],
                        self.grid_pos[x][y][1][1],
                        self.grid_pos[x][y][1][0],
                    )
            #update du canvas
            canvas.update()
            time.sleep(0.1)
        print(direction)

In [ ]:
root = Tk()
snake_game = Snake_game(root)
root.mainloop()

25
dict_keys([0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500])
26
25
[[(0, 0), (20, 20)], [(20, 0), (40, 20)], [(40, 0), (60, 20)], [(60, 0), (80, 20)], [(80, 0), (100, 20)], [(100, 0), (120, 20)], [(120, 0), (140, 20)], [(140, 0), (160, 20)], [(160, 0), (180, 20)], [(180, 0), (200, 20)], [(200, 0), (220, 20)], [(220, 0), (240, 20)], [(240, 0), (260, 20)], [(260, 0), (280, 20)], [(280, 0), (300, 20)], [(300, 0), (320, 20)], [(320, 0), (340, 20)], [(340, 0), (360, 20)], [(360, 0), (380, 20)], [(380, 0), (400, 20)], [(400, 0), (420, 20)], [(420, 0), (440, 20)], [(440, 0), (460, 20)], [(460, 0), (480, 20)], [(480, 0), (500, 20)]]
